<a href="https://colab.research.google.com/github/seismosmsr/machine_learning/blob/main/kaggle_api_in_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing the [Kaggle API](https://github.com/Kaggle/kaggle-api) in Colab

Just to ensure we've got our requirements met. This also works if you choose to run things on the kaggle back end (code for pushing notebook to auto submit from kaggle at bottom of this script).

In [ ]:
#THis is a new important line

In [1]:
!pip install kaggle
!conda install -y gdown

/bin/bash: conda: command not found


# Authenticating with Kaggle using kaggle.json

Navigate to https://www.kaggle.com. Then go to the [Account tab of your user profile](https://www.kaggle.com/me/account) and select Create API Token. This will trigger the download of kaggle.json, a file containing your API credentials.

Then run the cell below to upload kaggle.json to your Colab runtime.

In [2]:
# from google.colab import files
import gdown
#This is my personal kaggle json. If you run this and don't switch it out, you'll be running as me.
!gdown --id 1sD1x-nf2nXNNDFD3zdPKvOKM2wGSWcSP
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Downloading...
From: https://drive.google.com/uc?id=1sD1x-nf2nXNNDFD3zdPKvOKM2wGSWcSP
To: /content/kaggle.json
100% 69.0/69.0 [00:00<00:00, 94.2kB/s]


# Using the Kaggle API

For a more complete list of what you can do with the API, visit https://github.com/Kaggle/kaggle-api.

## Downloading a dataset

In [3]:
!kaggle competitions download -c covid-19-risk-2022

 98% 289M/296M [00:02<00:00, 124MB/s]
100% 296M/296M [00:02<00:00, 135MB/s]


Unzip the data and take a first glance.

In [4]:
!unzip train.csv.zip

!unzip test.csv.zip

!unzip train_small.csv.zip

unzip:  cannot find or open train.csv.zip, train.csv.zip.zip or train.csv.zip.ZIP.
unzip:  cannot find or open test.csv.zip, test.csv.zip.zip or test.csv.zip.ZIP.
unzip:  cannot find or open train_small.csv.zip, train_small.csv.zip.zip or train_small.csv.zip.ZIP.


In [5]:
import pandas as pd 

train_small = pd.read_csv('/content/train_small.csv')
# train = pd.read_csv('/content/train.csv')
# test = pd.read_csv('/content/test.csv')
train_small.head()

FileNotFoundError: ignored

So we've got a case_month, res_state, state_fips_code, res_county, county_fips_code, age_group, sex, race, ethnicity, lots more stuff. Some issues I can think of right away and which we've gone over in class, are the completeness of the data. A couple of approaches I've taken to these issues in the past are usually types of imputation. My initial thinking was to just randomly replace missing data with valid data. Another option would be to use clustering or random forest to create 'informed' imputed postiions. A third option would be to just not use the missing data and to try and maybe group 'types' of missing data together, then maximise the training set by 'grouping' the classes. 

Any which way, the method that we use to deal with NA's is going to be important. If we use a pure random imputation, this ..may.. be able to keep us unbiased (makes some assumptions about underlying distribution of missing data), but we definately stand a chance of losing information if we can't some how randomize the way the data is missing. One option might be to instead of only randomly imputing to that data once, we could do it many many times. If we do this many times, we could actually re-use some of our training data. We might think of this as similar to 'fuzzing' our data in deep learning (basic data augmetation).

If we use a systematic approach (like train an algorithm to impute for us), one issue we'll face is that it could bias our new 'imputed' training dataset. What I mean by this, is that our NA's may not be randomly distributed throughout our training dataset. Some hospitals or agencys may get some data and not others. Maybe some states are better or worse at their quality of reporting. If we rely on an algorthmic approach, we'll effectively be baking whatever biases exist in the meta context of our data into our data. We don't neccessarily want to do that.

Finally, we could also find a way to 'ignore' NA's while losing as little data as possible. In this approach, we could look at where data is and where data isnt, and see if we can find 'groups' of data that are missing some data, but not others. We could try doing this in a column-wise, row-wise, or column-by-row wise manner. Basically, we can see if we can find from the presense of missing data, some meta groups where most of the data is in-tact. Whichever approach we take, we would end up on different column/ row combinations of data and would be training multiple models on the back and that can then be reconfigured into an ensemble model.

I'm planning on focusing primarily on the data clean-up and filtering to get improvements on performance in this exercise.

Read in the data

In [ ]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

So lets take a look at our data.

In [ ]:
train.isnull().describe()

So even in our first row we can see what the degree of missing data looks like. The first row is missing information on age, sex, race, ethnicity. Not training on these data would be less than desireable.

In [ ]:
train.iloc[:1]

So if we look for null data, we find that most rows are missing at least some data. A minimally complete list would be composed of 'case_month', 'process', 'labconfirmed_yn'. Even our response variable, 'death_yn' is missing data.

In [ ]:
train.isnull().describe()

We'll also want to deal with other issues the data may have, specifically, there are several rows that have 'confusing' or confused data. We should also take the time to standarsize our data, but also set any very uncommon values to NA where appropriate. Effectively, we neeed to define a data dictionary to support us through the rest of this investigation.

In [ ]:
# for i in train.columns:
#   print('Unique items '+i)
#   print(train[i].unique())

So I'm going to ignore the correctness of County and State. I still want to use these data, but I can't know if spellings are all 100% correct, or at least, I don't think its worthe the time. Age group seems pretty well formed. The 'sex' column mayneed some work, but we can see that race has the same classes for alternative ways data couold be missing. We'll have to make a choice as to how to handle 'true' missing data.

Other columns with issues appear to be exposure_yn, lab_confirmed_yn, symptomatic_yn, basically all of the columns. Some of the issues are that they arent stanadardized into either float, string, or booleans. Also, there are some mixes and some text 'nulls' peppered in there. We'll need to address all of these before procceding.

Another consideration as part of this process, is that this NA processing and filtering has to happen on both our test and validation sets. We'll need to keep this in mind in how we implement our solution.


In [ ]:
#This is a weird one because we know they were all exposed, so not-knowing not
#really a nan
import numpy as np

train['exposure_yn'][train['exposure_yn'].isnull()] = 0.0
train['exposure_yn'][train['exposure_yn']== 1] = 1.0
# train['exposure_yn'] = train['exposure_yn'].astype('float')


train['labconfirmed_yn'][train['labconfirmed_yn']== 1] = 1.0
train['labconfirmed_yn'][train['labconfirmed_yn']== 0] = 0.0
# train['labconfirmed_yn'] = train['labconfirmed_yn'].astype('float')

train['symptomatic_yn'][train['symptomatic_yn']== '1'] = 1.0
train['symptomatic_yn'][train['symptomatic_yn']== '0'] = 0.0
# train['symptomatic_yn'] = train['symptomatic_yn'].astype('float')



train['hosp_yn'][train['hosp_yn']== 1] = 1.0
train['hosp_yn'][train['hosp_yn']== 0] = 0.0
# train['hosp_yn'] = train['hosp_yn'].astype('float')

train['icu_yn'][train['icu_yn']== '1'] = 1.0
train['icu_yn'][train['icu_yn']== '0'] = 0.0
train['icu_yn'][train['icu_yn']== 'nul'] = np.nan
# train['icu_yn'] = train['icu_yn'].astype('float')


In [ ]:
for i in train.columns:
  print('Unique items '+i)
  print(train[i].unique())

Ok, at least now our data is some what standardized and we can look for clustering among our NA. So that we can remember what tha looks like:

In [ ]:
train.isnull().describe()

We'll need to convert classes to booleans, so we're going to drop some redundant columns

In [ ]:
train.columns

In [ ]:
na_cluster_test = train[['case_month', 'res_state', 'res_county',
       'age_group', 'sex', 'race', 'ethnicity',
       'case_positive_specimen_interval', 'case_onset_interval', 'process',
       'exposure_yn', 'labconfirmed_yn', 'symptomatic_yn', 'hosp_yn', 'icu_yn',
       'death_yn', 'underlying_conditions_yn']]

na_cluster_test = train.isnull()
na_cluster_test = na_cluster_test.astype(int)

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(5, random_state=0)
kmeans.fit(na_cluster_test)
identified_clusters = kmeans.fit_predict(na_cluster_test)
identified_clusters

So what we're looking for here is any strong or weak groupings where its almost all one or the other, effectively, autocorrelation between NA's in some columns.

In [ ]:
na_cluster_test.groupby(identified_clusters).sum() / na_cluster_test.sum()

Whats interesting here, is that most of the clusters are pretty evenly split. However, there appears to be one cluster which sticks out and contains all the NA's from res_state, state_fips codes, age, sex, most of race, and most of ethnicity. Most of the other columns are more spread out between the clusters. It would like make sense then to split our modeling. We can make one model that predicts based on res_state, state_fips codes, age, sex, most of race, and ethnicity, since we can expect most of those columns to be complete where the others are complete. We can then place the rest of the columns in another model. In this way we can minimize the total amount of imputation we have to rely on, should we choose to rely on that. We can also consider not doing imputation and just running the two models where we have data.

We'll need a method for imputting random features into the dataset

In [ ]:
def randomiseMissingData(df):
    import random
    "randomise missing data for DataFrame (within a column)"
    # df = df2.copy()
    for col in df.columns:
        data = df[col]
        mask = data.isnull()
        samples = random.choices( data[~mask].values , k = mask.sum() )
        df.loc[mask,col] = samples
    return df

In [ ]:
train.columns

In [ ]:
train_fill = train[identified_clusters != 2]
train_fill['group'] = 'train'
test['group'] = 'test'
test['death_yn'] = 0
small_cluster_train = train_fill.append(test)

In [ ]:
# small_cluster_train = train[identified_clusters != 2]
small_cluster_train = small_cluster_train[['case_month','res_state','age_group','sex','race','ethnicity','death_yn','group']]
small_cluster_train	 = randomiseMissingData(small_cluster_train)
small_cluster_train = pd.get_dummies(small_cluster_train)	 
small_cluster_train = small_cluster_train[train['death_yn'].notnull()]

In [ ]:
small_cluster_train.columns

In [ ]:
# small_cluster_train['group_train']
train_filled = small_cluster_train[small_cluster_train['group_train'] == 1]
test_filled = small_cluster_train[small_cluster_train['group_train'] == 0]

In [ ]:
y = train_filled['death_yn']
del train_filled['death_yn']
del test_filled['death_yn']

In [ ]:
print(len(y))
print(len(small_cluster_train))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

tree = DecisionTreeClassifier()
bag = BaggingClassifier(tree, n_estimators=100, max_samples=.8,
                        random_state=1)

bag.fit(train_filled, y)

In [ ]:
test_y_pred = bag.predict(test_filled)

In [ ]:
# Create submission file.
submission = pd.DataFrame(test_y_pred, columns=['prediction']) # Create new dataframe.
submission['Id'] = submission.index  # Kaggle expects two columns: Id, prediction.
submission.to_csv('sample_submission.csv', index=False)

import matplotlib.pylab as plt
plt.hist(test_y_pred, bins=100);

So you can immediately notice some pretty big differences between the two dataset. Firstly, more counties and states in training than test. Probably a true random sample (which could mean that some rare counties or combinations of groups don't exist in the validation set). Same with sex, pretty big difference between the two. However thats a little weirder because there so many. The two ratios should be much closer than that.

In [ ]:
submission
!kaggle competitions submit covid-19-risk-2022 -f sample_submission.csv -m 'Column Coherence'

In [ ]:
# Example code for training model and creating submission file.
# Author: Peter Sadowski Jan 22 2022
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

# Load training data.
df_train = pd.read_csv('./train_small.csv.zip') # Can read from zip files directly.
df_train = df_train.replace({'death_yn':{np.nan:0}}) # Assume no info means survived.
y = df_train['death_yn']

# Load test data.
df_test = pd.read_csv('./test.csv.zip')

# Encode state variable as one-hot. 
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
enc.fit(df_train[['res_state']])
state_train = enc.transform(df_train[['res_state']])
state_test = enc.transform(df_test[['res_state']])

# Combine this with whether patient went to ICU.
X = np.concatenate([df_train[['icu_yn']]==1, state_train], axis=1)
X_test = np.concatenate([df_test[['icu_yn']] == 1, state_test], axis=1)

# Make predictions based on whether patient went to ICU, and their state.
model = LogisticRegression()
model.fit(X,y)
ypred = model.predict_proba(X_test)[:,1]
print(f'Model coefficients: {model.coef_}')

# Create submission file.
submission = pd.DataFrame(ypred, columns=['prediction']) # Create new dataframe.
submission['Id'] = submission.index  # Kaggle expects two columns: Id, prediction.
submission.to_csv('sample_submission.csv', index=False)

import matplotlib.pylab as plt
plt.hist(ypred, bins=100);

In [ ]:
submission
!kaggle competitions submit covid-19-risk-2022 -f sample_submission.csv -m 'Heres Johnny'


## Uploading a Colab notebook to Kaggle Kernels

Bear with us, as this is a little round-about...

### Downloading a notebook from Colab

To download from Colab, use **File** | **Download .ipynb**

In [ ]:
# user = "ics435"
# repo = "ps1-numpy-seismosmsr"
# src_dir = "master"
# pyfile = "kaggle_api_in_colab.ipynb"
# raw_git = 'https://raw.githubusercontent.com/seismosmsr/machine_learning/main/kaggle_api_in_colab.ipynb'

# url = f"{raw_git}"

# !wget --no-cache --backups=1 {url} -o submission.ipynb

### Then upload the notebook to your Colab runtime

In [ ]:
# # uploaded = files.upload()
# notebook_path = '/content/kaggle_api_in_colab.ipynb'

In [ ]:
# uploaded = files.upload()
# notebook_path = list(uploaded.keys())[0]

In [ ]:
# !mkdir -p export
# !mv $notebook_path export/
# !kaggle kernels init -p export

In [ ]:
# import re
# import random
# your_kaggle_username = 'Aron Boettcher'
# notebook_title = 'Test Kernel ' + str(random.randint(1,100))
# new_kernel_slug = re.sub(r'[^a-z0-9]+', '-', notebook_title.lower())
# notebook_path = 'kaggle_api_in_colab.ipynb'

In [ ]:
# # Documented here: https://github.com/Kaggle/kaggle-api/wiki/Kernel-Metadata
# metadata = '''
# {
#   "id": "%s/%s",
#   "title": "%s",
#   "code_file": "%s",
#   "language": "python",
#   "kernel_type": "notebook",
#   "is_private": "true",
#   "enable_gpu": "false",
#   "enable_internet": "true",
#   "dataset_sources": [],
#   "competition_sources": [],
#   "kernel_sources": []
# }
# ''' % (your_kaggle_username, new_kernel_slug, notebook_title, notebook_path)

In [ ]:
# !echo '$metadata' > export/kernel-metadata.json
# !cat export/kernel-metadata.json

In [ ]:
# !kaggle kernels push -p export

In [ ]:
# the functions:
def stratified_sample(df, strata, size=None, seed=None, keep_index= True):
    '''
    It samples data from a pandas dataframe using strata. These functions use
    proportionate stratification:
    n1 = (N1/N) * n
    where:
        - n1 is the sample size of stratum 1
        - N1 is the population size of stratum 1
        - N is the total population size
        - n is the sampling size
    Parameters
    ----------
    :df: pandas dataframe from which data will be sampled.
    :strata: list containing columns that will be used in the stratified sampling.
    :size: sampling size. If not informed, a sampling size will be calculated
        using Cochran adjusted sampling formula:
        cochran_n = (Z**2 * p * q) /e**2
        where:
            - Z is the z-value. In this case we use 1.96 representing 95%
            - p is the estimated proportion of the population which has an
                attribute. In this case we use 0.5
            - q is 1-p
            - e is the margin of error
        This formula is adjusted as follows:
        adjusted_cochran = cochran_n / 1+((cochran_n -1)/N)
        where:
            - cochran_n = result of the previous formula
            - N is the population size
    :seed: sampling seed
    :keep_index: if True, it keeps a column with the original population index indicator
    
    Returns
    -------
    A sampled pandas dataframe based in a set of strata.
    Examples
    --------
    >> df.head()
    	id  sex age city 
    0	123 M   20  XYZ
    1	456 M   25  XYZ
    2	789 M   21  YZX
    3	987 F   40  ZXY
    4	654 M   45  ZXY
    ...
    # This returns a sample stratified by sex and city containing 30% of the size of
    # the original data
    >> stratified = stratified_sample(df=df, strata=['sex', 'city'], size=0.3)
    Requirements
    ------------
    - pandas
    - numpy
    '''
    population = len(df)
    size = __smpl_size(population, size)
    tmp = df[strata]
    tmp['size'] = 1
    tmp_grpd = tmp.groupby(strata).count().reset_index()
    tmp_grpd['samp_size'] = round(size/population * tmp_grpd['size']).astype(int)

    # controlling variable to create the dataframe or append to it
    first = True 
    for i in range(len(tmp_grpd)):
        # query generator for each iteration
        qry=''
        for s in range(len(strata)):
            stratum = strata[s]
            value = tmp_grpd.iloc[i][stratum]
            n = tmp_grpd.iloc[i]['samp_size']

            if type(value) == str:
                value = "'" + str(value) + "'"
            
            if s != len(strata)-1:
                qry = qry + stratum + ' == ' + str(value) +' & '
            else:
                qry = qry + stratum + ' == ' + str(value)
        
        # final dataframe
        if first:
            stratified_df = df.query(qry).sample(n=n, random_state=seed).reset_index(drop=(not keep_index))
            first = False
        else:
            tmp_df = df.query(qry).sample(n=n, random_state=seed).reset_index(drop=(not keep_index))
            stratified_df = stratified_df.append(tmp_df, ignore_index=True)
    
    return stratified_df



def stratified_sample_report(df, strata, size=None):
    '''
    Generates a dataframe reporting the counts in each stratum and the counts
    for the final sampled dataframe.
    Parameters
    ----------
    :df: pandas dataframe from which data will be sampled.
    :strata: list containing columns that will be used in the stratified sampling.
    :size: sampling size. If not informed, a sampling size will be calculated
        using Cochran adjusted sampling formula:
        cochran_n = (Z**2 * p * q) /e**2
        where:
            - Z is the z-value. In this case we use 1.96 representing 95%
            - p is the estimated proportion of the population which has an
                attribute. In this case we use 0.5
            - q is 1-p
            - e is the margin of error
        This formula is adjusted as follows:
        adjusted_cochran = cochran_n / 1+((cochran_n -1)/N)
        where:
            - cochran_n = result of the previous formula
            - N is the population size
    Returns
    -------
    A dataframe reporting the counts in each stratum and the counts
    for the final sampled dataframe.
    '''
    population = len(df)
    size = __smpl_size(population, size)
    tmp = df[strata]
    tmp['size'] = 1
    tmp_grpd = tmp.groupby(strata).count().reset_index()
    tmp_grpd['samp_size'] = round(size/population * tmp_grpd['size']).astype(int)
    return tmp_grpd


def __smpl_size(population, size):
    '''
    A function to compute the sample size. If not informed, a sampling 
    size will be calculated using Cochran adjusted sampling formula:
        cochran_n = (Z**2 * p * q) /e**2
        where:
            - Z is the z-value. In this case we use 1.96 representing 95%
            - p is the estimated proportion of the population which has an
                attribute. In this case we use 0.5
            - q is 1-p
            - e is the margin of error
        This formula is adjusted as follows:
        adjusted_cochran = cochran_n / 1+((cochran_n -1)/N)
        where:
            - cochran_n = result of the previous formula
            - N is the population size
    Parameters
    ----------
        :population: population size
        :size: sample size (default = None)
    Returns
    -------
    Calculated sample size to be used in the functions:
        - stratified_sample
        - stratified_sample_report
    '''
    if size is None:
        cochran_n = round(((1.96)**2 * 0.5 * 0.5)/ 0.02**2)
        n = round(cochran_n/(1+((cochran_n -1) /population)))
    elif size >= 0 and size < 1:
        n = round(population * size)
    elif size < 0:
        raise ValueError('Parameter "size" must be an integer or a proportion between 0 and 0.99.')
    elif size >= 1:
        n = size
    return n
